In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sps
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import pickle
import os.path

from collections import defaultdict

# I. Класс реализованного метода

In [223]:
class UserBasedRecommender():
    def __init__(self, path_to_orders, path_to_order_products_train, decomposition_dim):
        '''
        :param path_to_orders: путь к исходному датафрейму с keggle orders.csv
        :param path_to_order_products_train: путь к новому тренировочному датафрейму, полученному в п.II ниже
        :param decomposition_dim: размерность декомпозиции
        '''
        self.path_to_orders = path_to_orders
        self.path_to_order_products_train = path_to_order_products_train
        self.n_comp = decomposition_dim
        
        self.ok_status = True
        check, message = self.check_input_params()
        
        if not check:
            self.ok_status = False
            print(message)
    
    def check_input_params(self):
        if not os.path.isfile(self.path_to_orders):
            return False, 'File doesnt exist'
        if not os.path.isfile(self.path_to_order_products_train):
            return False, 'File doesnt exist'
        if not isinstance(self.n_comp, int) or not self.n_comp > 0:
            return False, 'decomposition_dim should be positive integer value'
        
        return True, 'Ok'
            
    def read_data(self):
        if not self.ok_status:
            return 'There were some problems'
        
        self.orders = pd.read_csv(self.path_to_orders)
        self.order_products_train = pd.read_csv(self.path_to_order_products_train)
    
    def prepare_data(self):
        '''
        создает таблицу user-item для дальнейших манипуляций
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        self.merged = pd.merge(self.orders, self.order_products_train, on='order_id', how='right')
        self.merged = self.merged[['order_id', 'user_id', 'product_id']]
        
        self.user_id_num = {key: value for value, key in enumerate(self.merged['user_id'].unique(), start=0)}
        self.product_id_num = {key: value for value, key in enumerate(self.merged['product_id'].unique(), start=0)}
        self.product_num_id = {value: key for value, key in enumerate(self.merged['product_id'].unique(), start=0)}
        
        self.merged['user_num'] = self.merged['user_id'].transform(lambda x: self.user_id_num[x])
        self.merged['product_num'] = self.merged['product_id'].transform(lambda x: self.product_id_num[x])
        self.merged['buy'] = 1
        
        self.user_item_matrix = sps.coo_matrix((self.merged.buy, (self.merged.user_num, self.merged.product_num)))
        
    def decompose(self):
        '''
        декомпозиция user-item_matrix с размерностью n_comp
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        self.model = NMF(n_components=self.n_comp, init='random', random_state=0)
        self.user_matrix = self.model.fit_transform(self.user_item_matrix)
        self.product_matrix = self.model.components_
    
    def get_similarity(self, top_n_size):
        '''Каждому юзеру сопоставляет id n-схожих юзеров
        :param top_n_size: сколько схожих пользователей по убыванию схожести надо сохранить
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        
        self.similar_users_df = pd.DataFrame()
        self.similar_users_weight_df = pd.DataFrame()
            
        size = 100
        users_cnt = len(self.user_matrix)
        add = 1 if users_cnt % size > 0 else 0
        parts_cnt = users_cnt//size + add

        for i in tqdm(range(parts_cnt)):
            start = size * i
            end = size * i + size
                
            user_to_others_similarity = cosine_similarity(Y=self.user_matrix, X=self.user_matrix[start:end])
                
            sim_users_df = pd.DataFrame(np.argsort(-user_to_others_similarity)).loc[:, :top_n_size]
            sim_users_weight_df = pd.DataFrame(-np.sort(-user_to_others_similarity)).loc[:, :top_n_size]
                
            self.similar_users_df = pd.concat([self.similar_users_df, sim_users_df], ignore_index=True)
            self.similar_users_weight_df = pd.concat([self.similar_users_weight_df, sim_users_weight_df], ignore_index=True)
    
    def save_similarity_df(self, path_users, path_weights):
        if not self.ok_status:
            return 'There were some problems'
        
        self.similar_users_df.to_csv(path_users, index=False)
        self.similar_users_weight_df.to_csv(path_weights, index=False)
        
    def set_similarity_df(self, path_to_df_users, path_to_df_weights):
        '''
        Если уже есть посчитанная матрица, то можем использовать и обойти долгий кусок обучения в fit(),
        указав параметр loaded_df=True
        :param path_to_df: путь до csv файла с учебным датафреймом
        '''
        if os.path.isfile(path_to_df_users) and os.path.isfile(path_to_df_weights):
            self.similar_users_df = pd.read_csv(path_to_df_users)
            self.similar_users_weight_df = pd.read_csv(path_to_df_weights)
        else:
            # оформить ошибки по-человечески - нужен же возврат к True
            # self.ok_status = False
            return 'File doesnt exist'
    
    def fit(self, path_to_save_users=False, path_to_save_weights=False, loaded_df=False, top_n_size=5):
        '''
        :param self:
        :param path_to_save: путь для сохранения матрицы сх. юзеров и весов (названия дефолтные)
        :param loaded_df: bool - True если матрица уже была загружена и пересчитывать заново ее не нужно;
                          default=False
        :param top_n_size: размер топ схожих пользователей 
        :return: информация в случае ошибки 
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        self.top_n_size = top_n_size
        
        self.read_data()     # читает необходимые df
        self.prepare_data()  # создание таблицы user-item с необх. полями и разметками
        if not loaded_df:
            self.decompose()     # декомпозиция матрицы на 2 матрицы размерности n
            self.get_similarity(self.top_n_size)
            if path_to_save_users and path_to_save_weights:
                self.save_similarity_df(path_to_save_users, path_to_save_weights)
    
    def predict(self, user_id=False, sim_users_count=False, users_count=False):
        '''
        :param self:
        :param user_id:  id юзера, для которого считаем предсказание покупок, если False, то считаем всех
        :param sim_users_count: если нужно учесть не весь топ схожих юзеров, а только его часть, 
                                равную значению данного параметра
        :param users_count: если нужно посчитать не для одного/всех юзеров, а для части, 
                                равной значению данного параметра
        :return: сообщение в случае ошибки; в случае успеха - словарь {user_id: products_id}
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        if not sim_users_count or sim_users_count > self.top_n_size:
            sim_users_count = self.top_n_size
        
        if user_id:
            if user_id in self.user_id_num:
                users_to_check = [user_id] 
            else:
                return 'There is no user with this id'
        else:
            users_to_check = [*self.user_id_num][:users_count] if users_count else [*self.user_id_num]
            
        self.predicted_products = dict()
            
        users_purchases = pd.Series.sparse.from_coo(self.user_item_matrix)
        
        # проход по всем юзерам, для которых хотим получить предсказание покупок
        for user_id in tqdm(users_to_check):
            user_num = self.get_user_num(user_id)
            
            # номера и веса схожих c user_id пользователей
            sim_users_nums = self.similar_users_df.loc[user_num].values
            sim_users_weights = self.similar_users_weight_df.loc[user_num].values

            d = defaultdict(int)
            
            # проход по всем юзерам схожим с предсказываемым
            for sim_user_num, sim_user_weight in zip(sim_users_nums[:sim_users_count], sim_users_weights[:sim_users_count]):

                sim_user_purchases = users_purchases[sim_user_num].index
                
                # к каждому купленному схожим юзером продукту прибавляем его вес относительно предсказываемого
                for product in sim_user_purchases:
                        d[product] += sim_user_weight

            d_ids = defaultdict(int)

            # ключами были номера продуктов - заменим на id
            for key in d:
                d_ids[self.product_num_id[key]] = d[key]
                
            self.predicted_products[user_id] = [*dict(sorted(d_ids.items(), key=lambda x: x[1], reverse=True))]
        return self.predicted_products
    
    def get_coo(self):
        return self.user_item_matrix
    
    def get_user_num(self, user_id):
        return self.user_id_num[user_id]
    
    def prepare_validation_data(self, path_to_valid_df):
        '''
        создает таблицу user-item для дальнейших манипуляций
        :param path_to_valid_df: путь до csv таблицы с валидационным датафреймом
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        self.order_products_valid = pd.read_csv(path_to_valid_df)
        
        self.merged_valid = pd.merge(self.orders, self.order_products_valid, on='order_id', how='right')
        self.merged_valid = self.merged_valid[['order_id', 'user_id', 'product_id']]
        
        self.user_id_num_valid = {key: value for value, key in enumerate(self.merged_valid['user_id'].unique(), start=0)}
        self.product_id_num_valid = {key: value for value, key in enumerate(self.merged_valid['product_id'].unique(), start=0)}
        self.product_num_id_valid = {value: key for value, key in enumerate(self.merged_valid['product_id'].unique(), start=0)}
        
        self.merged_valid['user_num'] = self.merged_valid['user_id'].transform(lambda x: self.user_id_num_valid[x])
        self.merged_valid['product_num'] = self.merged_valid['product_id'].transform(lambda x: self.product_id_num_valid[x])
        self.merged_valid['buy'] = 1
        
        self.user_item_matrix_valid = sps.coo_matrix((self.merged_valid.buy, (self.merged_valid.user_num, self.merged_valid.product_num)))
        
    def get_data_to_check_metrics(self, users_count=False, sim_users_count=False):
        '''
        :param count_users: для скольких юзеров посчитаем метрики
        :param sim_users_count: сколько юзеров из топа схожих будут учитываться для предсказания покупок
        '''
        predicted_purchases = {}
        actual_purchases = {}
        
        predicted = self.predict(sim_users_count=sim_users_count, users_count=users_count)
        
        users_purchases_valid = pd.Series.sparse.from_coo(self.user_item_matrix_valid)
        
        for user_id in predicted:
            if user_id in self.user_id_num_valid:
                user_num_in_valid_df = self.user_id_num_valid[user_id]
                user_actual_purchases_nums = users_purchases_valid[user_num_in_valid_df].index
                    
                actual = [self.product_num_id_valid[product_num] for product_num in user_actual_purchases_nums]
            
                predicted_purchases[user_id] = predicted[user_id]
                actual_purchases[user_id] = actual
            
            else:
                print('No user with id: ', user_id)
        
        return actual_purchases, predicted_purchases
        
            
        

# II. Пропишем все пути и сконструируем датасеты мечты

In [99]:
import pickle

# необходимые файлы с исходными датасетами с keggle
path_data_train = 'D:/Projects/Anaconda/EDA/data/order_products__train.csv'
path_data_prior = 'D:/Projects/Anaconda/EDA/data/order_products__prior.csv'
path_data_orders = 'D:/Projects/Anaconda/EDA/data/orders.csv'
    
# [куда запишем]/[где уже лежат] файлы с индексами
train_path = 'data_split/train_indices.pickle'
valid_path = 'data_split/validation_indices.pickle'
test_path = 'data_split/test_indices.pickle'

# куда запишем файлы с новым разделением
df_train_path = 'data_split/train_data.csv'
df_valid_path = 'data_split/validation_data.csv'
df_test_path = 'data_split/test_data.csv'

### Если файлов с индексами нет

In [102]:
def _split_indices(grouped_ratings, retriever):
    return np.concatenate(grouped_ratings.apply(retriever).values)

def split(path_to_orders):
    orders = pd.read_csv(path_to_orders)
    grouper = orders.sort_values('order_number').groupby('user_id')
    train_indices = _split_indices(
        grouper,
        lambda user_ratings: user_ratings[:int(user_ratings.shape[0] * 0.5)].index.values)
    
    validation_indices = _split_indices(
        grouper,
        lambda user_ratings: user_ratings.iloc[int(user_ratings.shape[0] * 0.5):
                                               int(user_ratings.shape[0] * 0.75)].index.values)
    
    test_indices = _split_indices(
        grouper,
        lambda user_ratings: user_ratings.iloc[int(user_ratings.shape[0] * 0.75):].index.values)
    
    return train_indices, validation_indices, test_indices


train_indices, validation_indices, test_indices = split(path_data_orders)

# save results
with open(train_path, 'wb') as out:
    pickle.dump(train_indices, out)

with open(valid_path, 'wb') as out:
    pickle.dump(validation_indices, out)

with open(test_path, 'wb') as out:
    pickle.dump(test_indices, out)

### Файлы с индексами готовы - теперь скомпонуем и запишем в файл все необходимое

In [103]:
def read_and_concat_same_data(df_train_path, df_prior_path, df_orders_path):
    
    df_train = pd.read_csv(df_train_path)
    df_prior = pd.read_csv(df_prior_path)
    df_orders = pd.read_csv(df_orders_path)
    
    order_products_full = pd.concat([df_train, df_prior])
    
    return order_products_full, df_orders


def get_split_df(train_path, valid_path, test_path, df_orders, df_order_products_full):
    
    with open(train_path, 'rb') as input:
        train_indices = pickle.load(input)

    with open(valid_path, 'rb') as input:
        validation_indices = pickle.load(input)

    with open(test_path, 'rb') as input:
        test_indices = pickle.load(input)
        
    train_order_ids = df_orders.loc[train_indices]['order_id'].values
    validation_order_ids = df_orders.loc[validation_indices]['order_id'].values
    test_order_ids = df_orders.loc[test_indices]['order_id'].values
    
    train_df = df_order_products_full.loc[train_order_ids]
    validation_df = df_order_products_full.loc[validation_order_ids]
    test_df = df_order_products_full.loc[test_order_ids]
    
    return train_df, validation_df, test_df


# считается что уже имеем 3 файла с индексами
df_order_products_full, df_orders = read_and_concat_same_data(path_data_train, path_data_prior, path_data_orders)
train_df, validation_df, test_df = get_split_df(train_path, valid_path, test_path, df_orders, df_order_products_full)

# write new df's to file:
train_df.to_csv(df_train_path, index=False)
validation_df.to_csv(df_valid_path, index=False)
test_df.to_csv(df_test_path, index=False)

# IV. Посчитаем с новым алгоритмом

### Пример с обучением

In [ ]:
# файл, куда по необходимости загрузим результаты
path_to_res_users = 'data/res_top_30_dim_15_users.csv'
path_to_res_weights = 'data/res_top_30_dim_15_weights.csv'
# необходимые файлы: 
# 1. исходный из keggle orders
path_to_orders = 'data/orders.csv'

# 2. созданный-смерженный как надо методами из п.II
df_train_path = 'data_split/train_data.csv'

# размерность для декомпозиции
n_comp = 15

# top_n
top_n_size = 30

# создаем экземпляр класса с вышеописанными параметрами
Recommender = UserBasedRecommender(path_to_orders, df_train_path, n_comp)

Recommender.fit(path_to_save_users=path_to_res_users, path_to_save_weights=path_to_res_weights, top_n_size=top_n_size)

### Пример без обучения (просто загрузка рез-тов)

In [184]:
# файл, куда по необходимости загрузим результаты
path_to_res_users = 'data/res_top_30_dim_15_users.csv'
path_to_res_weights = 'data/res_top_30_dim_15_weights.csv'
# необходимые файлы: 
# 1. исходный из keggle orders
path_to_orders = 'data/orders.csv'

# 2. созданный-смерженный как надо методами из п.II
df_train_path = 'data_split/train_data.csv'

# размерность для декомпозиции
n_comp = 15

# top_n
top_n_size = 30

# создаем экземпляр класса с вышеописанными параметрами
Recommender = UserBasedRecommender(path_to_orders, df_train_path, n_comp)

# Загружаем уже посчитанные результаты
Recommender.set_similarity_df(path_to_res_users, path_to_res_weights)

Recommender.fit(loaded_df=True, top_n_size=30)

In [149]:
# посчитаем для 5 юзеров и у каждого учтем только топ-3 схожих
Recommender.predict(users_count=1, sim_users_count=3)

D:\Soft\Anaconda\lib\site-packages\ipykernel_launcher.py:167: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


{1: [27845,
  23826,
  26405,
  39657,
  46149,
  32478,
  43154,
  3798,
  35336,
  44683,
  2120,
  33065,
  23504,
  30162]}

In [155]:
# посчитаем для юзера с id 125 и учтем только топ-3 схожих
Recommender.predict(sim_users_count=3, user_id=10)

D:\Soft\Anaconda\lib\site-packages\ipykernel_launcher.py:167: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


{10: [17795,
  28535,
  9339,
  40604,
  46979,
  16797,
  34969,
  10749,
  25230,
  22825,
  40706,
  13212,
  27104,
  45664,
  30489,
  11782,
  41220,
  16857,
  13512,
  23165,
  32299,
  36695,
  15011,
  43014,
  28986,
  47380,
  47591,
  22035,
  47526,
  23541,
  42625,
  36865,
  9871,
  13829,
  28842,
  31506,
  48720,
  29650,
  21616,
  45007,
  45066,
  17553,
  23765,
  33731,
  8277,
  49131,
  7781,
  37947,
  28465,
  43122,
  43933,
  17579,
  26521,
  46359,
  20015,
  34423,
  2954,
  39694,
  38300,
  42265,
  20940,
  13424,
  8571,
  4724,
  1957]}

# Метрички

In [ ]:
# Рассматривать большое число соседей не имеет смысла, так как результат либо тот же либо сильно хуже
# 5-15 довольно оптимальные числа и по времени занимает меньше, чем предсказывать покупки по 100 схожим юзерам

# Размерность декомпозиции тоже влияет, с dim=50 результаты в целом получше, чем с 10, 
# но считалось это в разы дольше - может имеет смысл взять около 25

In [185]:
# готовим данные для проверки метрик
Recommender.prepare_validation_data('data_split/validation_data.csv')

In [213]:
# получаем данные предсказанное/реально купленное
predicted, actual = Recommender.get_data_to_check_metrics(users_count=1000, sim_users_count=5)

D:\Soft\Anaconda\lib\site-packages\ipykernel_launcher.py:167: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



No user with id:  4
No user with id:  13
No user with id:  22
No user with id:  30
No user with id:  66
No user with id:  72
No user with id:  81
No user with id:  92
No user with id:  116
No user with id:  120
No user with id:  139
No user with id:  178
No user with id:  198
No user with id:  201
No user with id:  225
No user with id:  240
No user with id:  242
No user with id:  260
No user with id:  271
No user with id:  275
No user with id:  282
No user with id:  285
No user with id:  314
No user with id:  317
No user with id:  326
No user with id:  344
No user with id:  365
No user with id:  377
No user with id:  383
No user with id:  384
No user with id:  395
No user with id:  400
No user with id:  408
No user with id:  431
No user with id:  436
No user with id:  441
No user with id:  456
No user with id:  461
No user with id:  472
No user with id:  477
No user with id:  483
No user with id:  518
No user with id:  525
No user with id:  549
No user with id:  561
No user with id:  

In [179]:
# AP@k
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

# MAP@K
def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [214]:
mapk(predicted.values(), actual.values(), k=10)
# среднее по 1000 пользователей
# если учитывались 2 схожих результат был 0.048
# для 5 схожих 0.043
# для 10-20... <0.034

# то есть чем больше схожих юзеров учитываем, тем ниже становится точность (хотя она и так не сильно большая)

0.04320114060241818

# Summary

Ускорены методы fit(), get_similarity(), predict(), get_data_to_check_metrics()

fit():
   + вынесена self.decompose() для выполнения только в том случае, если нужно полное обучение модели
   
get_similarity():
   + ранее считал для размерности декомпозиции 10 около 3 часов, сейчас это занимает порядка полутора часов (ускорилось благодаря счету не каждого юзера отдельно, а по 100 за раз, те матричные операции выполняются быстрее)
   + теперь считает как номера пользователей, так и их веса (те теперь считается и сохраняется 2 датафрейма, как рез-т обучения)
   + использован другой способ сортировки данных (полагаю это тоже ускорило процесс)
   
predict():
   + Из цикла вынесена операция from_coo, занимавшая порядка секунды, вытащить из уже посчитанной структуры необходимые данные по индексу уже сущие копейки
   + Не считает заново веса, так как они теперь считаются в get_similarity()
   + Исходя из параметров может посчитать для одного юзера с опеределенным id/для всех юзеров/для части юзеров, начиная с 1; также есть параметр, указывающий сколько из топа схожих юзеров надо учесть
   + По итогу на весь датасет, на который ранее требовалось 4500 часов, сейчас требуется порядка 7 часов, если уменьшить число пользователей, по которым сделаем итоговую оценку, то управится со всеми данными примерно за полтора часа для 5 и за 2.5 часа для 10.
   
get_data_to_check_metrics():
   + Ускорен засчет оптимизации вышеописанных методов
   
Можем рассчитать любой топ пользователей, но метрика оказывается больше, если учитываем меньшее число схожих юзеров в predict()